## Task Manager

Problem Statement:
In today’s world, individuals often need to keep track of various tasks in a structured
way. You are tasked with building a Task Manager that allows users to manage their
tasks. The system should include user authentication, meaning each user has to log
in with a username and password. Once logged in, users can create, view, update,
and delete their tasks. Each user’s tasks should be stored separately, and only the
authenticated user can access their tasks.
Objectives:
1. Design and implement a user authentication system (login and registration)
2. Create a task management system that allows users to:
a. Add, view, mark as completed, and delete tasks
3. Use file handling to store user credentials and tasks persistently
4. Create an interactive menu-driven interface to manage tasks

Hints:
- Build a file to store credentials in the following format:
    ```
    username, password
    Mark, 12398234
    Mike, 866123
    Becky, 1271823
    ```
- build functions that execute each step in the assignment


In [71]:
from getpass import getpass
from datetime import datetime
import os
import csv
import hashlib
import uuid
import json

In [72]:
def get_users():
    user_db = []
    #check if user database exists
    try:
        if not os.path.exists('user_db.csv'):
            with open('user_db.csv', 'w') as file:
                #write the header
                csv.writer(file).writerow(['username', 'password'])

        with open('user_db.csv', 'r') as file:
            reader = csv.reader(file)
            for row in reader:
                #skip the header
                if row[0] == 'username':
                    continue
                username, password = row
                user_db.append({'username': username, 'password': password})

        return user_db
    except FileNotFoundError as e:
        print('User database is down')
        raise e

def get_user_by_username(username):
    user_db = get_users()

    for user in user_db:
        if user['username'] == username:
            # an object of user
            return user

    return None

def register_user():
    # use getpass library
    username = input('Enter username:')
    password = getpass('Enter password:')

    # check if the user already exists
    if get_user_by_username(username):
        print('User already exists')
        return

    # if the user does not exist
    # Create hash using SHA-256 (returns hexadecimal string)
    password_hash = hashlib.sha256(password.encode()).hexdigest()

    # add the user to the database
    with open('user_db.csv', 'a') as file:
        try:
            # write to next line in the csv the new user using csv.writer
            csv.writer(file).writerow([username, password_hash])
        except Exception as e:
            print('Error saving user to database')
            raise e
    print('User registered successfully')

    user = get_user_by_username(username)
    return user


In [73]:
'''
Login:
o Create a function to prompt the user for their username and
password, validate the credentials by comparing them with the stored
data, and grant access to the task manager upon successful login
'''

def login():
    username = input('Enter username:')
    password = getpass('Enter password:')

    user = get_user_by_username(username)

    if user is None:
        print('Invalid username')
        return

    if user['password'] == hashlib.sha256(password.encode()).hexdigest():
        print('Login successful')
        return user
    else:
        print('Invalid password')
        return None


In [74]:
#  Add a Task:
# • Create a function that prompts the user for a task description. Assign a
# unique task ID and set the status to Pending
# • Store the task in a file, and confirm that the task was added

def get_tasks_db():
    tasks_db = {}

    try:
        if not os.path.exists('task_db.json'):
            with open('task_db.json', 'w') as file:
                json.dump(tasks_db, file)
                return tasks_db

        with open('task_db.json', 'r') as file:
            tasks_db = json.load(file)

        return tasks_db

    except (FileNotFoundError, Exception) as e:
        print('Task database is down')
        raise e

def update_tasks_db(tasks_db):
    with open('task_db.json', 'w') as file:
        try:
            # make sure file is empty before writing back the taskdb to the db file
            file.truncate()
            json.dump(tasks_db, file)
        except Exception as e:
            print('Error saving tasks to database')
            raise e

def save_new_task(username, task, status):
    tasks_db = get_tasks_db()
    user_tasks = tasks_db.get(username, None)

    if user_tasks is None:
        user_tasks = []

    user_tasks.append(task)
    tasks_db[username] = user_tasks

    update_tasks_db(tasks_db)

def add_task(username):
    description = input('(Required) Enter task description: ')
    task_uid = str(uuid.uuid4())[:8]

    #check if the description is empty
    if description == '':
        print('Description is required')
        return
    
    tasks_db = get_tasks_db()
    user_tasks = tasks_db.get(username, None)

    if user_tasks is None:
        user_tasks = []

    user_tasks.append({'task_id': task_uid, 'description': description , 'status': 'pending'})
    tasks_db[username] = user_tasks

    update_tasks_db(tasks_db)

    print('Task added successfully!')

def update_task(username, task):
    tasks_db = get_tasks_db()
    user_tasks = tasks_db.get(username, None)

    if user_tasks is None or len(user_tasks) == 0:
        print('You have no tasks')
        return

    for t in user_tasks:
        if t['task_id'] == task['task_id']:
            t['description'] = task['description']
            t['status'] = task['status']
            break

    update_tasks_db(tasks_db)


# Mark a Task as Completed:
# • Create a function that allows the user to select a task by its ID and update
# its status to Completed

def mark_task_as_completed(username):
    tasks_db = get_tasks_db()
    user_tasks = tasks_db.get(username, None)

    if user_tasks is None or len(user_tasks) == 0:
        print('You have no tasks')
        return
    
    # print all the pending tasks using view_tasks function
    print('\n\n Choose a pending task to mark as completed using the task ID')
    view_tasks(username, 'pending')

    # ask the user to select a task by its ID
    task_id = input('Enter the pending task ID to mark as completed:')
    task_found = False

    for task in user_tasks:
        if task['task_id'] == task_id:
            task['status'] = 'completed'
            update_task(username, task)
            task_found = True
            break

    if not task_found:
        print('Task not found')
    else:
        print('Task marked as completed successfully!')

# Delete a Task:
# • Create a function that allows the user to select a task by its ID and delete
# it from their task list
def delete_task(username):
    tasks_db = get_tasks_db()
    user_tasks = tasks_db.get(username, None)

    if user_tasks is None or len(user_tasks) == 0:
        print('You have no tasks')
        return

    # print all the tasks using view_tasks function
    view_tasks(username, 'all')

    task_id = input('Enter the task ID to delete:')
    task_found = False

    for task in user_tasks:
        if task['task_id'] == task_id:
            user_tasks.remove(task)
            tasks_db[username] = user_tasks
            update_tasks_db(tasks_db)
            task_found = True
            break

    if not task_found:
        print('Task not found')
    else:
        print('Task deleted successfully!')

# View Tasks:
# • Create a function to retrieve and display all tasks for the logged-in user
# • Each task should show the task ID, description, and status (Pending or
# Completed)

def view_tasks(username, category):
    tasks_db = get_tasks_db()
    user_tasks = tasks_db.get(username, None)

    if user_tasks is None or len(user_tasks) == 0:
        print('You have no tasks')
        return

    if category == 'pending':
        print('Pending Tasks')
        print('-'*30)
        for task in user_tasks:
            if task['status'] == 'pending':
                #print in table format
                print(f'| {task["task_id"]:<10} | {task["description"]:<20} | Pending |')
    elif category == 'completed':
        print('Completed Tasks')
        print('-'*30)
        for task in user_tasks:
            if task['status'] == 'completed':
                print(f'| {task["task_id"]:<10} | {task["description"]:<20} | Completed |')
    else:
        # 'print all the user tasks pending first and then completed'
        for task in user_tasks:
            print(f'| {task["task_id"]:<10} | {task["description"]:<20} | {task["status"]} |')


In [75]:
def unauth_user_handler(choice):
    user = None
    if choice == '1':
      user = register_user()
    elif choice == '2':
        user = login()

    return user

def auth_user_handler(logged_in_user, choice):
    if choice == '1':
        add_task(logged_in_user['username'])
    elif choice == '2':
        view_tasks(logged_in_user['username'], 'pending')
    elif choice == '3':
        view_tasks(logged_in_user['username'], 'all')
    elif choice == '4':
        mark_task_as_completed(logged_in_user['username'])
    elif choice == '5':
        delete_task(logged_in_user['username'])

# Create an interactive menu-driven interface to manage tasks
def run_task_manager():
    is_logged_in = False
    logged_in_user = None
    choice = None

    while True:
        if not is_logged_in:
            choice = input('''
                        Please select an option: \n
                        1. Register \n
                        2. Login \n
                        Enter your choice (1-2): '''
                    )
            logged_in_user = unauth_user_handler(choice)
            if logged_in_user is not None:
                is_logged_in = True
        else:
            choice = input('''
                        Please select an option:
                        1. Add Task
                        2. View Pending Tasks
                        3. View All Tasks
                        4. Mark Task as Completed
                        5. Delete Task
                        6. Logout
                        Enter your choice (1-6): '''
                    )
            if choice == '6':
                break
            auth_user_handler(logged_in_user, choice)


In [77]:
run_task_manager()


Login successful
You have no tasks
You have no tasks
You have no tasks
You have no tasks
